<a href="https://colab.research.google.com/github/EAName/yelp-reviews/blob/main/Cleaning_Business_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import json
import os

import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import statistics
import numpy as np

%matplotlib inline

In [51]:
# First drop yelp_dataset into your drive (Add shortcut to Drive)
business_json_path = '/content/gdrive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

In [52]:
#keep only restaurant category
df_b = df_b[df_b['categories'].str.contains(
              'restaurant|Restaurant|RESTAURANT',
              case=False, na=False)]

In [53]:
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


# Elaine's DF Clean of Business


Drop Canada and other states

In [6]:
# look at rows with NC, CO, HI, MT, XMS to see if they are mis-labeled or if they should be dropped

df_b.loc[df_b['state'].isin(["NC", "CO", "HI", "MT", "XMS"])]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
9101,em474WuTw7-OkhPfX4EohA,IHOP,702 Blowing Rock Rd,Boone,NC,28607,36.378837,-86.463779,2.0,29,0,"{'Alcohol': ''none'', 'RestaurantsTakeOut': 'F...","Breakfast & Brunch, Coffee & Tea, Food, Americ...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."
9126,1H9WYGFJ4AFwXq7WCOOMtg,Two Elk Restaurant,12000 Skyline Dr,Vail,CO,81657,32.094854,-110.773491,3.5,12,0,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...","Buffets, Restaurants",None
12258,bw90rjFL02A_8rVE34m5Lg,Cosmetic Solutions,1010 Lula Lula Dr,Lula Lula,HI,90261,38.816486,-90.321750,3.5,17,1,"{'BusinessAcceptsCreditCards': 'False', 'Accep...","Cosmetic Surgeons, Pizza, Doctors, Beauty & Sp...",None
48919,gCF0QQWzI9lDouhLfTbYCA,MudMan Food Truck,,Kalispell,MT,59901,27.789678,-82.680746,5.0,6,0,"{'RestaurantsTakeOut': 'True', 'BikeParking': ...","Restaurants, American (Traditional), Food Truc...","{'Tuesday': '17:0-19:30', 'Wednesday': '11:0-1..."
98685,IvQs2hhXKZ4NgZhVZ5jjjQ,Pumpernickles Catering,95a Mount Pleasant,Liverpool,XMS,L3 5TB,38.673049,-90.423156,4.0,5,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Caterers, Breakfast & Brunch, Convenience Stor...",None


Remove these 5 states since they don't belong to a market 

In [54]:
#Drop rows for restaurants in outlier states (4 rows) & Canada

#dfb_restaurants1.loc[dfb_restaurants1['state'].isin(["NC", "CO", "HI", "MT", "XMS"])]

removestates = ["NC", "CO", "HI", "MT", "XMS", "AB"]

df_b = df_b[~ df_b['state'].isin(removestates)]

In [55]:
df_b.shape

(49871, 14)

Total restaurants remaining should be 49,871

Normalize cities and states into DMA markets with market column
For CA cities, I checked that Reno, NV should only contain NV rows and all CA rows belong to Santa Barbara

In [56]:
#Test that CA zipcodes belonging to Reno DMA are not in the df

renoziptest = ["93512", "93517", "93529", "93541", "93546", "95646", "96009", "96056", "96068", "96107", "96109", "96113", "96114", "96117", "96119", "96120", "96121", "96123", "96127", 
               "96128", "96130", "96132", "96133", "96136", "96150", "96151", "96152", "96154", "96155", "96156", "96157", "96158"]

In [57]:
renotestdf = df_b[df_b['postal_code'].isin(renoziptest)]
renotestdf.shape

(0, 14)

In [58]:
#Get list of all states

df_b.state.value_counts()

PA    12644
FL     8732
TN     4353
MO     4248
IN     4150
LA     3641
NJ     3343
AZ     2675
NV     1675
ID     1304
CA     1161
IL      983
DE      962
Name: state, dtype: int64

In [59]:
#create market dictionary

market = {'PA': 'Philadelphia', 'DE': 'Philadelphia', 'NJ': 'Philadelphia', 'FL': 'Tampa', 'IN': 'Indianapolis', 'TN': 'Nashville', 'AZ': 'Tucson', 'LA': 'New Orleans', 'MO': 'St. Louis', 'IL': 'St. Louis',
          'NV': 'Reno', 'CA': 'Santa Barbara', 'ID': 'Boise'}

In [13]:
df_b['market']= df_b['state'].map(market)

Look at all the restaurant categories by most frequent

In [60]:
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

In [61]:
pd.set_option('display.max_rows', None)  # or 1000

In [62]:
df_explode.categories.value_counts().head(200)

Restaurants                  49853
Food                         14818
Nightlife                     8355
Sandwiches                    8136
Bars                          7988
American (Traditional)        7943
Pizza                         6779
Fast Food                     6151
Breakfast & Brunch            6023
American (New)                5989
Burgers                       5392
Mexican                       4542
Italian                       4395
Coffee & Tea                  3839
Seafood                       3457
Salad                         3000
Event Planning & Services     2999
Chinese                       2967
Chicken Wings                 2880
Cafes                         2595
Delis                         2344
Caterers                      2015
Specialty Food                1917
Bakeries                      1829
Desserts                      1761
Sports Bars                   1742
Japanese                      1695
Barbeque                      1653
Sushi Bars          

Remove rows to narrow "Restaurant" Definition to Sit-down restaurants

In [63]:
#Create DF to isolate the rows we want to remove
#Please add more as needed

df_bfast = df_b[df_b['categories'].str.contains('Fast Food|Bakery|Bakeries|Food Trucks|Street Vendors|Coffee| Coffee & Tea|Caterers|Ice Cream & Frozen Yogurt|Juice Bars & Smoothies|Convenience Stores|Food Stands|Gas Stations|Creperies|Beauty & Spa|International Grocery|Gelato|Butcher|Shaved Ice',
                                                                                 case=False, na=False)]
df_bfast.shape

(14581, 14)

In [64]:
df_b = df_b.drop(df_bfast.index)
df_b.shape

(35290, 14)

Total rows remaining should be 35,290

In [65]:
#Check whether top fast food names are still in the df

fast_list = ["McDonald's", "Subway", "Taco Bell", "Burger King", "Wendy's", "Wawa","Domino's Pizza", "Pizza Hut", "Papa John's Pizza", "Jimmy John's",
             "KFC", "Chick-fil-A", "Popeyes Louisiana Kitchen", "Chipotle Mexican Grill", "Starbucks", "Panera Bread", "Sonic Drive-In", "Arby's",
             "Panda Express", "Waffle House", "IHOP", "Jack in the Box", "Jersey Mike's Subs" , "Steak ’n Shake", "Five Guys", "Dairy Queen" , "QDOBA Mexican Eats", 
             "Little Caesars Pizza", "Hardee's", "Wingstop", "Firehouse Subs", "Dairy Queen Grill & Chill", "Buffalo Wild Wings", "Checkers", "China Wok", 
             "White Castle", "Imo's Pizza", "Papa Murphy's", "PrimoHoagies", "Saladworks", "Marco's Pizza", "QuikTrip", "Philly Pretzel Factory", "Boston Market"
              "Moe's Southwest Grill", "Church's Chicken", "Culver's", "Rally's", "Cracker Barrel Old Country Store", "Tropical Smoothie Café", 
             "Hungry Howie's Pizza & Subs", "McAlister's Deli", "Jet's Pizza", "Hooters", "Little Caesars", "St. Louis Bread", "Friendly's", 
             "Long John Silver's", "Dickey's Barbecue Pit", "Zaxby's Chicken Fingers & Buffalo Wings", "Captain D's", "Chuck E. Cheese", "Carl's Jr",
             "MOD Pizza", "Einstein Bros. Bagels", "Honey Baked Ham Company", "eegee's", "Manhattan Bagel", "Raising Cane's Chicken Fingers", "Noodles & Company", 
             "Smashburger", "Penn Station East Coast Subs", "Tijuana Flats",  "Westshore Pizza", "Zoes Kitchen", "Port of Subs", "Quiznos", 
             "Shake Shack", "Fazoli's"]

In [66]:
print(len(df_b[df_b['name'].isin(fast_list)]))

1803


There are still 1803 rows with the chains in fast_list, representing 5% of the total businesses. We may need to check if we can delete from Pizza and Sandwich category. The only issue with deleting Pizza is that it may delete "Italian" also.

Larger Chains that are still kept within the Total restaurants list to provide comparison for sit-down restaurants:



*   Applebee's Grill + Bar
*   Chili's
* Denny's
* First Watch
* Outback Steakhouse
* China King - there is a chain but there are also independents
* Olive Garden Italian Restaurant
* LongHorn Steakhouse
* Red Lobster
* Ruby Tuesday
* TGI Fridays
* Bob Evans
* Red Robin Gourmet Burgers and Brews
* Beef 'O'Brady's
* Texas Roadhouse
* Carrabba's Italian Grill
* China House
* O'Charley's Restaurant & Bar
* Bonefish Grill
* New China







# Rabia's DF Clean of Business

### One Hot Encode Attributes column

In [67]:
df_b = df_b.join(pd.DataFrame(df_b['attributes'].to_dict()).T)

In [68]:
pd.set_option('display.max_columns', None)

In [69]:
df_b.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Caters,Alcohol,RestaurantsAttire,RestaurantsDelivery,RestaurantsTakeOut,HasTV,NoiseLevel,BusinessAcceptsCreditCards,OutdoorSeating,BusinessParking,Ambience,RestaurantsPriceRange2,GoodForKids,WiFi,RestaurantsReservations,RestaurantsGoodForGroups,BikeParking,GoodForMeal,DogsAllowed,HappyHour,WheelchairAccessible,RestaurantsTableService,BusinessAcceptsBitcoin,Smoking,Music,GoodForDancing,CoatCheck,BestNights,BYOB,ByAppointmentOnly,DriveThru,Corkage,BYOBCorkage,RestaurantsCounterService,Open24Hours,AgesAllowed,AcceptsInsurance,DietaryRestrictions
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,True,u'full_bar',u'casual',False,True,True,u'average',True,True,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",1,True,u'free',False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,"{'RestaurantsReservations': 'False', 'Restaura...","American (Traditional), Restaurants, Diners, B...","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",NaN,'none','casual',True,True,True,NaN,True,False,"{'garage': None, 'street': False, 'validated':...","{'touristy': None, 'hipster': None, 'romantic'...",1,True,u'no',False,True,False,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",NaN,'full_bar','casual',True,True,False,u'average',True,True,"{u'valet': False, u'garage': None, u'street': ...","{'touristy': False, 'hipster': False, 'romanti...",2,False,'free',True,True,NaN,"{'dessert': True, 'latenight': None, 'lunch': ...",False,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",True,u'none',u'casual',None,True,True,u'quiet',True,None,"{'garage': False, 'street': True, 'validated':...","{'touristy': False, 'hipster': False, 'romanti...",1,True,u'no',False,False,True,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,6625 E 82nd St,Indianapolis,IN,46250,39.904320,-86.053080,4.0,20,1,"{'RestaurantsDelivery': 'True', 'RestaurantsTa...","Steakhouses, Asian Fusion, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, Italian","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",True,'full_bar','casual',True,True,True,u'average',True,True,"{'garage': False, 'street': False, 'validated'...","{u'divey': False, u'hipster': False, u'casual'...",2,True,u'free',True,True,True,"{'dessert': None, 'latenight': False, 'lunch':...",False,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,1160 Gallatin Pike S,Nashville,TN,37115,36.248570,-86.719986,4.0,6,0,"{'RestaurantsReservations': 'False', 'Restaura...","

### Clean the categories

In [70]:
df_b.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'Caters', 'Alcohol',
       'RestaurantsAttire', 'RestaurantsDelivery', 'RestaurantsTakeOut',
       'HasTV', 'NoiseLevel', 'BusinessAcceptsCreditCards', 'OutdoorSeating',
       'BusinessParking', 'Ambience', 'RestaurantsPriceRange2', 'GoodForKids',
       'WiFi', 'RestaurantsReservations', 'RestaurantsGoodForGroups',
       'BikeParking', 'GoodForMeal', 'DogsAllowed', 'HappyHour',
       'WheelchairAccessible', 'RestaurantsTableService',
       'BusinessAcceptsBitcoin', 'Smoking', 'Music', 'GoodForDancing',
       'CoatCheck', 'BestNights', 'BYOB', 'ByAppointmentOnly', 'DriveThru',
       'Corkage', 'BYOBCorkage', 'RestaurantsCounterService', 'Open24Hours',
       'AgesAllowed', 'AcceptsInsurance', 'DietaryRestrictions'],
      dtype='object')

In [71]:
all_attributes = ['Caters', 'Alcohol',
       'RestaurantsAttire', 'RestaurantsDelivery', 'RestaurantsTakeOut',
       'HasTV', 'NoiseLevel', 'BusinessAcceptsCreditCards', 'OutdoorSeating',
       'BusinessParking', 'Ambience', 'RestaurantsPriceRange2', 'GoodForKids',
       'WiFi', 'RestaurantsReservations', 'RestaurantsGoodForGroups',
       'BikeParking', 'GoodForMeal', 'DogsAllowed', 'HappyHour',
       'WheelchairAccessible', 'RestaurantsTableService',
       'BusinessAcceptsBitcoin', 'Smoking', 'Music', 'GoodForDancing',
       'CoatCheck', 'BestNights', 'BYOB', 'ByAppointmentOnly', 'DriveThru',
       'Corkage', 'BYOBCorkage', 'RestaurantsCounterService', 'Open24Hours',
       'AgesAllowed', 'AcceptsInsurance', 'DietaryRestrictions']

In [72]:
# sub attributes that can be cleaned up later. I could not find a way to hot encode these without getting error messages.
# Dietary restrictions has sub-attributes but will be manually cleaned
sub_attributes =  ['BusinessParking', 'Ambience', 'GoodForMeal', 'Music', 'BestNights']

In [73]:
no_sub_attributes = ['Caters', 'Alcohol',
       'RestaurantsAttire', 'RestaurantsDelivery', 'RestaurantsTakeOut',
       'HasTV', 'NoiseLevel', 'BusinessAcceptsCreditCards', 'OutdoorSeating', 
       'RestaurantsPriceRange2', 'GoodForKids',
       'WiFi', 'RestaurantsReservations', 'RestaurantsGoodForGroups',
       'BikeParking', 'DogsAllowed', 'HappyHour',
       'WheelchairAccessible', 'RestaurantsTableService',
       'BusinessAcceptsBitcoin', 'Smoking', 'GoodForDancing',
       'CoatCheck', 'BYOB', 'ByAppointmentOnly', 'DriveThru',
       'Corkage', 'BYOBCorkage', 'RestaurantsCounterService', 'Open24Hours',
       'AgesAllowed', 'AcceptsInsurance', 'DietaryRestrictions']

In [79]:
for column in no_sub_attributes:
  print(column,': ',df_b[column].unique().tolist())

Caters :  ['True', nan, 'False']
Alcohol :  ["u'full_bar'", "'none'", "'full_bar'", "u'none'", nan, "u'beer_and_wine'", "'beer_and_wine'"]
RestaurantsAttire :  ["u'casual'", "'casual'", nan, "u'formal'", "'dressy'", "u'dressy'", "'formal'"]
RestaurantsDelivery :  ['False', 'True', nan]
RestaurantsTakeOut :  ['True', nan, 'False']
HasTV :  ['True', 'False', nan]
NoiseLevel :  ["u'average'", nan, "u'quiet'", "'average'", "u'loud'", "'quiet'", "'very_loud'", "u'very_loud'", "'loud'"]
BusinessAcceptsCreditCards :  ['True', nan, 'False']
OutdoorSeating :  ['True', 'False', nan]
RestaurantsPriceRange2 :  ['1', '2', nan, '3', '4']
GoodForKids :  ['True', 'False', nan]
WiFi :  ["u'free'", "u'no'", "'free'", nan, "'no'", "'paid'", "u'paid'"]
RestaurantsReservations :  ['False', 'True', nan]
RestaurantsGoodForGroups :  ['True', 'False', nan]
BikeParking :  [nan, 'False', 'True']
DogsAllowed :  [nan, 'False', 'True']
HappyHour :  [nan, 'True', 'False']
WheelchairAccessible :  [nan, 'True', 'False

In [75]:
df_clean = df_b

In [76]:
# Some missing values are represented by different values such as nan, None, 'None'
# Combine into one category

for column in no_sub_attributes:
  df_clean[column] = df_clean[column].replace(['none','None', None],np.NaN)

In [90]:
# manually fix other columns by combining some categories

df_clean['Alcohol'] = df_clean['Alcohol'].replace(["u'full_bar'", "'none'", "'full_bar'", "u'none'", "u'beer_and_wine'", "'beer_and_wine'"],
                                                  ['full_bar', 'None', 'full_bar', 'None', 'beer_and_wine', 'beer_and_wine'])

df_clean['RestaurantsAttire'] = df_clean['RestaurantsAttire'].replace(["u'casual'", "'casual'", "u'formal'", "'dressy'", "u'dressy'", "'formal'"],
                                                  ['casual', 'casual', 'formal', 'dressy', 'dressy', 'formal'])

df_clean['NoiseLevel'] = df_clean['NoiseLevel'].replace(["u'average'", "u'quiet'", "'average'", "u'loud'", "'quiet'", "'very_loud'", "u'very_loud'", "'loud'"],
                                                  ['average', 'quiet', 'average', 'loud', 'quiet', 'very_loud', 'very_loud', 'loud'])

df_clean['WiFi'] = df_clean['WiFi'].replace(["u'free'", "u'no'", "'free'", "'no'", "'paid'", "u'paid'"],
                                                  ['free', 'no', 'free', 'no', 'paid', 'paid'])

df_clean['Smoking'] = df_clean['Smoking'].replace(["u'no'", "u'outdoor'", "u'yes'", "'no'", "'outdoor'"],
                                                  ['no', 'outdoor', 'yes', 'no', 'outdoor'])

df_clean['BYOBCorkage'] = df_clean['BYOBCorkage'].replace(["'yes_free'", "'no'", "'yes_corkage'", "u'yes_free'", "u'yes_corkage'", "u'no'"],
                                                  ['yes_free', 'no', 'yes_corkage', 'yes_free', 'yes_corkage', 'no'])

df_clean['AgesAllowed'] = df_clean['AgesAllowed'].replace(["u'allages'", "u'21plus'"],
                                                  ['allages', '21plus'])

In [92]:
#Manually clean dietary restrictions
df_clean['DietaryRestrictions'] = df_clean['DietaryRestrictions'].replace(["{'dairy-free': False, 'gluten-free': False, 'vegan': False, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': True}", "{'dairy-free': False, 'gluten-free': True, 'vegan': False, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': False}", "{'dairy-free': False, 'gluten-free': False, 'vegan': True, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': False}", "{'dairy-free': False, 'gluten-free': False, 'vegan': True, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': True}", "{'dairy-free': True, 'gluten-free': True, 'vegan': True, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': True}", "{'dairy-free': False, 'gluten-free': False, 'vegan': True, 'kosher': False, 'halal': True, 'soy-free': False, 'vegetarian': True}"],
                                                  ['vegetarian','gluten_free', 'vegan_vegetarian', 'vegan_vegetarian',
                                                   'dairy-free_gluten-free_vegan_vegetarian', 'vegan_vegetarian_halal'])

In [98]:
df_clean['DietaryRestrictions'].value_counts()

gluten_free                                5
vegetarian                                 3
vegan_vegetarian                           3
dairy-free_gluten-free_vegan_vegetarian    1
vegan_vegetarian_halal                     1
Name: DietaryRestrictions, dtype: int64

Only 13 rows have data for dietary restrictions. Not going to one-hot encode because most are missing

In [100]:
# Confirm that attribute columns are clean
for column in no_sub_attributes:
  print(column,': ',df_clean[column].unique().tolist())

Caters :  ['True', nan, 'False']
Alcohol :  ['full_bar', 'None', nan, 'beer_and_wine']
RestaurantsAttire :  ['casual', nan, 'formal', 'dressy']
RestaurantsDelivery :  ['False', 'True', nan]
RestaurantsTakeOut :  ['True', nan, 'False']
HasTV :  ['True', 'False', nan]
NoiseLevel :  ['average', nan, 'quiet', 'loud', 'very_loud']
BusinessAcceptsCreditCards :  ['True', nan, 'False']
OutdoorSeating :  ['True', 'False', nan]
RestaurantsPriceRange2 :  ['1', '2', nan, '3', '4']
GoodForKids :  ['True', 'False', nan]
WiFi :  ['free', 'no', nan, 'paid']
RestaurantsReservations :  ['False', 'True', nan]
RestaurantsGoodForGroups :  ['True', 'False', nan]
BikeParking :  [nan, 'False', 'True']
DogsAllowed :  [nan, 'False', 'True']
HappyHour :  [nan, 'True', 'False']
WheelchairAccessible :  [nan, 'True', 'False']
RestaurantsTableService :  [nan, 'True', 'False']
BusinessAcceptsBitcoin :  [nan, 'False', 'True']
Smoking :  [nan, 'no', 'outdoor', 'yes']
GoodForDancing :  [nan, 'False', 'True']
CoatCheck :

While most of these new columns are boolean attributes, a number of them are categorical, which need some special handling. We need to encode them so models can use them as features.

In [101]:
# columns with non-boolean categorical values:
cols_to_split = ['AgesAllowed', 'Alcohol', 'BYOBCorkage', 'NoiseLevel', 'RestaurantsAttire', 'Smoking', 'WiFi']
new_cat = pd.concat([pd.get_dummies(df_clean[col], prefix=col, prefix_sep='_') for col in cols_to_split], axis=1)
# keep all columns (not n-1) because 0's for all of them indicates that the data was missing (useful info)
df_clean = pd.concat([df_clean, new_cat], axis=1)
df_clean.drop(cols_to_split, inplace=True, axis=1)
df_clean.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Caters,RestaurantsDelivery,RestaurantsTakeOut,HasTV,BusinessAcceptsCreditCards,OutdoorSeating,BusinessParking,Ambience,RestaurantsPriceRange2,GoodForKids,RestaurantsReservations,RestaurantsGoodForGroups,BikeParking,GoodForMeal,DogsAllowed,HappyHour,WheelchairAccessible,RestaurantsTableService,BusinessAcceptsBitcoin,Music,GoodForDancing,CoatCheck,BestNights,BYOB,ByAppointmentOnly,DriveThru,Corkage,RestaurantsCounterService,Open24Hours,AcceptsInsurance,DietaryRestrictions,AgesAllowed_21plus,AgesAllowed_allages,Alcohol_None,Alcohol_beer_and_wine,Alcohol_full_bar,BYOBCorkage_no,BYOBCorkage_yes_corkage,BYOBCorkage_yes_free,NoiseLevel_average,NoiseLevel_loud,NoiseLevel_quiet,NoiseLevel_very_loud,RestaurantsAttire_casual,RestaurantsAttire_dressy,RestaurantsAttire_formal,Smoking_no,Smoking_outdoor,Smoking_yes,WiFi_free,WiFi_no,WiFi_paid
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,True,False,True,True,True,True,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",1,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,"{'RestaurantsReservations': 'False', 'Restaura...","American (Traditional), Restaurants, Diners, B...","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",NaN,True,True,True,True,False,"{'garage': None, 'street': False, 'validated':...","{'touristy': None, 'hipster': None, 'romantic'...",1,True,False,True,False,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",NaN,True,True,False,True,True,"{u'valet': False, u'garage': None, u'street': ...","{'touristy': False, 'hipster': False, 'romanti...",2,False,True,True,NaN,"{'dessert': True, 'latenight': None, 'lunch': ...",False,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",True,NaN,True,True,True,NaN,"{'garage': False, 'street': True, 'validated':...","{'touristy': False, 'hipster': False, 'romanti...",1,True,False,False,True,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
22,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,6625 E 82nd St,Indianapolis,IN,46250,39.904320,-86.053080,4.0,20,1,"{'RestaurantsDelivery': 'True', 'RestaurantsTa...","Steakhouses, Asian Fusion, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",NaN,True,True,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [103]:
# Replace boolean with numeric values
df_clean.replace({False: 0, True: 1}, inplace=True)

This is the clean dataset version 1. The missing attribute values have not been replaced but will need to be for future model development. 
The sub_attributes =  ['BusinessParking', 'Ambience', 'GoodForMeal', 'Music', 'BestNights'] need to one-hot encoded later.

In [107]:
df_clean.to_csv('/content/gdrive/MyDrive/yelp_dataset/yelp_business_clean_version1.csv')